In [2]:
import pandas as pd
import os

In [39]:
def load_data(main_dir, subdirectories):
    dataframes = []
    for subdir in subdirectories:
        for file in os.listdir(os.path.join(main_dir, subdir)):
            if file.endswith('.csv'):
                difficulty = 'easy' if 'easy' in file else 'hard'
                df = pd.read_csv(os.path.join(main_dir, subdir, file))
                print(df.shape)
                print(file)
                df['Difficulty'] = difficulty
                df['Model'] = subdir
                dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

main_dir = ""
subdirectories = ['llama3']
final_df = load_data(main_dir, subdirectories)

import re
import pandas as pd

def extract_answers(df):
    def extract_answer(name, string):
        if name in ["GSM8K_hard", "GSM8K_test"]:
            # Extract the last appearing digit, with or without comma separation
            match = re.findall(r'\d+(?:,\d+)*', string)
            if match:
                return match[-1].replace(',', '')
            else:
                return "Error: No digits found in the string."
        else:
            # Find the first option (A, B, C, D, or E) after "therefore", "so", or "thus"
            match = re.search(r'(?i)(therefore|so|thus).*?(?P<option>(?:(?:[Aa])|(?:\(?[Aa]\)?))|(?:(?:[Bb])|(?:\(?[Bb]\)?))|(?:(?:[Cc])|(?:\(?[Cc]\)?))|(?:(?:[Dd])|(?:\(?[Dd]\)?))|(?:(?:[Ee])|(?:\(?[Ee]\)?)))', string)
            if match:
                return re.sub(r'[()]', '', match.group('option')).upper()
            else:
                # Find the last appearing option
                match = re.findall(r'(?i)(?P<option>(?:(?:[Aa])|(?:\(?[Aa]\)?))|(?:(?:[Bb])|(?:\(?[Bb]\)?))|(?:(?:[Cc])|(?:\(?[Cc]\)?))|(?:(?:[Dd])|(?:\(?[Dd]\)?))|(?:(?:[Ee])|(?:\(?[Ee]\)?)))', string)
                if match:
                    return re.sub(r'[()]', '', match[-1]).upper()
                else:
                    return "Error: No valid options found in the string."

    for i in range(40):
        df[f'Final Answer_{i}'] = df.apply(lambda row: extract_answer(row['Name'], row[f'CoT_{i}']), axis=1)

    return df

llama3 = extract_answers(final_df)

llama3.to_csv('llama3_cleaned/full.csv', index=False)

(500, 124)
MathQA_easy.csv
(500, 124)
BigBench_easy.csv
(500, 124)
MathQA_hard.csv
(500, 124)
BigBench_hard.csv
(500, 124)
GSM8K_hard.csv
(500, 124)
GSM8K_easy.csv
